# New component

Type here a description for this component

## Wrapping Model

Allows your component to expose a service over REST.

To wrap your model [follow the instructions](https://docs.seldon.io/projects/seldon-core/en/v0.3.0/python/python_component.html) for your chosen language or toolkit.

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Model(object):
    """
    Model template. You may load your model and artifacts in __init__ using platiagro SDK.

    Visit https://platiagro.github.io/sdk/ to find all the details about the SDK.
    """

    def __init__(self, dataset: str = None, target: str = None):
        logger.info(f"dataset: {dataset}")
        logger.info(f"target: {target}")

        # Loads models and other artifacts using PlatIAgro SDK
        model = load_model()

        self.columns_names = ["col0", "col1", "col2", "col3"]

    def class_names(self):
        """The column names of the array that `predict` returns."""
        return self.columns_names

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Takes an array (numpy) X and feature_names and returns another array.

        Your return array should be at least 2-dimensional.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Dict of metadata.
        """
        # Builds a DataFrame from numpy.array.
        df = pd.DataFrame(X, columns=feature_names)

        X_predict = df.to_numpy()

        # start your code here...

        return X_predict

## API Contract

There are two sections:

- `features` : The feature array you intend to send in a request
- `targets` : The response you expect back

Each section has a list of definitions. Each definition consists of:

- `name` : String : The name of the feature
- `ftype` : one of CONTINUOUS, CATEGORICAL : the type of the feature
- `dtype` : One of FLOAT, INT : Required for ftype CONTINUOUS : What type of feature to create
- `values` : list of Strings : Required for ftype CATEGORICAL : The possible categorical values
- `range` : list of two numbers : Optional for ftype CONTINUOUS : The range of values (inclusive) that a continuous value can take
- `repeat` : integer : Optional value for how many times to repeat this value
- `shape` : array of integers : Optional value for the shape of array to coerce the values

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 7.0]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 4.0]
        },
        {
            "name": "PetalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 7.0]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 3.0]
        }
    ],
    "targets": [
        {
            "name": "Iris-setosa",
            "ftype": "continuous",
            "dtype": "FLOAT",
            "range": [0.0, 1.0]
        },
                {
            "name": "Iris-versicolor",
            "ftype": "continuous",
            "dtype": "FLOAT",
            "range": [0.0, 1.0]
        },
                {
            "name": "Iris-virginica",
            "ftype": "continuous",
            "dtype": "FLOAT",
            "range": [0.0, 1.0]
        }
    ]
}

## Test Deployment

Starts a service wrapping a Model, sends a request to the service, and shows the response.

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")